In [2]:
import numpy as np
from sklearn import datasets
import torch
import torch.nn as nn
import torch.optim as optim
import torchbnn as bnn
import matplotlib.pyplot as plt


In [3]:
TRAIN_SIZE = 2000
RANGE_x0 = 6
RANGE_x1 = 6

x_0 = torch.rand(TRAIN_SIZE)*2*RANGE_x0-RANGE_x0
x_1 = torch.rand(TRAIN_SIZE)*2*RANGE_x1-RANGE_x1
#y_0 = functy0
#y_1 = functy1
x_0, x_1 = torch.unsqueeze(x_0, dim=1), torch.unsqueeze(x_1, dim=1)
#y_0, y_1 = torch.unsqueeze(y_0, dim=1), torch.unsqueeze(y_1, dim=1)


In [4]:

y_0_clean = lambda x0, x1: 4*np.power(x0, 5) + 12*np.power(x1, 3) - 5

y_1_clean = lambda x0, x1: 3*np.power(x0, 4) - 7*np.power(x1, 2) + 9*x0*x1

clean_y0, clean_y1 = y_0_clean(x_0, x_1), y_1_clean(x_0, x_1)

max_y0_scale, max_y1_scale = clean_y0.max()*0.2, clean_y1.max()*0.2 #scale for noisy function

y_0_target = lambda x0, x1: y_0_clean(x0, x1) + (max_y0_scale*torch.rand(x0.size()) - max_y0_scale/2)

y_1_target = lambda x0, x1: y_1_clean(x0, x1) + (max_y1_scale*torch.rand(x0.size()) - max_y1_scale/2)


In [5]:
dropout_prob = 0.3
model = nn.Sequential(
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=2, out_features=1000),
    nn.ReLU(),
    nn.Dropout(dropout_prob),
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=1000, out_features=2),
)


In [6]:
mse_loss = nn.MSELoss()
kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
kl_weight = 0.01

optimizer = optim.Adam(model.parameters(), lr=0.01)

print(x_0.size(), x_1.size())

#creating training dataset

x_train = torch.cat((x_0, x_1), dim=1)

y_train = torch.cat((y_0_target(x_0, x_1), y_1_target(x_0, x_1)), dim=1)

print(x_train.size(), y_train.size())

#normalising dataset
normalise = lambda x: (x - torch.mean(x)) / (torch.std(x))

x_train[:,0], x_train[:,1] = normalise(x_train[:,0]), normalise(x_train[:,1])

y_train[:,0], y_train[:,1] = normalise(y_train[:,0]), normalise(y_train[:,1])

print(x_train.size())



for step in range(2000):
    pre_y = model(x_train)
    mse_y0 = mse_loss(pre_y[:, 0], y_train[:, 0])
    mse_y1 = mse_loss(pre_y[:, 1], y_train[:, 1]) 
    kl = kl_loss(model)
    cost = mse_y0 + mse_y1 + kl_weight*kl
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    
print('- MSE y0 : %2.2f ' % (mse_y0.item()))
print('- MSE y1 : %2.2f ' % (mse_y1.item()))


torch.Size([2000, 1]) torch.Size([2000, 1])
torch.Size([2000, 2]) torch.Size([2000, 2])
torch.Size([2000, 2])
- MSE y0 : 0.06 
- MSE y1 : 0.10 


In [7]:
TEST_SIZE = int(0.2*TRAIN_SIZE)

#create test dataset

x_0_test = torch.rand(TEST_SIZE)*2*RANGE_x0-RANGE_x0
x_1_test = torch.rand(TEST_SIZE)*2*RANGE_x1-RANGE_x1

x_0_test, x_1_test = torch.unsqueeze(x_0_test, dim=1), torch.unsqueeze(x_1_test, dim=1)

y_test = torch.cat((y_0_target(x_0_test,x_1_test), y_1_target(x_0_test, x_1_test)), dim=1)

x_test = torch.cat((x_0_test, x_1_test), dim=1)

#normalising dataset

x_test[:,0], x_test[:,1] = normalise(x_test[:,0]), normalise(x_test[:,1])

y_test[:,0], y_test[:,1] = normalise(y_test[:,0]), normalise(y_test[:,1])

print(x_test.size(), y_test.size())


models_result = np.array([model(x_test).data.numpy() for k in range(10000)])
#print(models_result.shape)
#models_result = models_result[:,:,0]
#print(models_result.shape)   
#models_result = models_result.T
#print(models_result.shape)  
#mean_values = np.array([models_result[i].mean() for i in range(len(models_result))])
#std_values = np.array([models_result[i].std() for i in range(len(models_result))])

#print(mean_values)
# Transpose the result to have dimensions [num_predictions, 500, 2]
models_result = models_result.transpose(0, 2, 1)

# Calculate mean and std separately for each column
mean_values = models_result.mean(axis=0)
std_values = models_result.std(axis=0)

# Transpose back to have dimensions [500, 2]
mean_values = mean_values.transpose(1, 0)
std_values = std_values.transpose(1, 0)

#print(mean_values)
#print(y_test)


torch.Size([400, 2]) torch.Size([400, 2])


In [8]:
mean_values_tensor = torch.tensor(mean_values, dtype=torch.float32)
test_mse_y0 = mse_loss(mean_values_tensor[:,0], y_test[:,0])
test_mse_y1 = mse_loss(mean_values_tensor[:,1], y_test[:,1])
print(f'TEST MSE for y0: {test_mse_y0.item():2.4f}')
print(f'TEST STD for y0: {std_values[:,0].mean():2.4f}')
print(f'TEST MSE for y1: {test_mse_y1.item():2.4f}')
print(f'TEST STD for y1: {std_values[:,1].mean():2.4f}')


TEST MSE for y0: 0.0408
TEST STD for y0: 0.1554
TEST MSE for y1: 0.0549
TEST STD for y1: 0.1727
